In [2]:
import cv2
import mediapipe as mp
import keyboard 
import numpy as np
import json
from KeyBindings import quit, readCoords,writeCoords
#init
#---------------------
cap= cv2.VideoCapture(0)

hands= mp.solutions.hands.Hands(static_image_mode=False,
                        max_num_hands=1,
                        min_tracking_confidence=0.5,
                        min_detection_confidence=0.5)

mpDraw = mp.solutions.drawing_utils

#---------------------------------------     
def ScaleAndRotateHand():
    
    pivotTrackedWritst = int( result.multi_hand_landmarks[0].landmark[0].x*w), int( result.multi_hand_landmarks[0].landmark[0].y*h)
    pivotTrackedFirstBone = int( result.multi_hand_landmarks[0].landmark[1].x*w), int( result.multi_hand_landmarks[0].landmark[1].y*h)
    #print(np.dot(pivotTracked[1]-pivotTracked[0],pivotOffsetImageRatioXY[1]-pivotOffsetImageRatioXY[0]))
    print(pivotTrackedFirstBone- pivotTrackedWritst)
#---------------------------------------   
# draw the standard hand from mediapipe
def drawHandMediaPipe(img, lm):
    h, w, _ = img.shape
    cx, cy = int(lm.x*w), int(lm.y*h)
    cv2.circle(img, (cx, cy), 3, (255, 0, 255)) 
    mpDraw.draw_landmarks(img, result.multi_hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS)
#---------------------------------------   
#Draws the saved gesture, the one from media pipe and compares both,
# TODO: THis function should be split in two for a single recognition part
def drawSavedHandGesture(id,h,w, recognizedColor):
    
     if len(savedCoords[0])<=2:
            return
     else:
        #Set the percentual offset for the wrist when the hand is scaled
        reciproce =rootBoone[2]/ savedCoords[0][2]
     # get the hand wrist as compare offset x,y,z and merge them to the xyz
        pivotPointX = (result.multi_hand_landmarks[0].landmark[1].x-(savedCoords[0][0]*reciproce))
        pivotPointY = (result.multi_hand_landmarks[0].landmark[1].y -(savedCoords[0][1]*reciproce))
        pivotPointZ = (result.multi_hand_landmarks[0].landmark[1].z -savedCoords[0][2])
        pivotXYZ = pivotPointX,pivotPointY,pivotPointZ
        #Draw the tracked handgesture as before but without the lines in between
        pivotTracked = int( result.multi_hand_landmarks[0].landmark[id].x*w), int( result.multi_hand_landmarks[0].landmark[id].y*h)
        # Calculate the savedCoordinates with the offset of the current projected gesture to match the hand wrist position independed of the screen
        savedOffset = np.dot(savedCoords[id],rootBoone[2]/savedCoords[0][2] )
        
        savedOffset =  [savedOffset[0]+pivotXYZ[0],  savedOffset[1]+pivotXYZ[1],  savedOffset[2]+pivotXYZ[2]]
       
        # For visual representation, it has to be redrawn in respect to the width and height ratio of the screen
        pivotOffsetImageRatio = [int (savedOffset[0]*w), int (savedOffset[1]*h), savedOffset[2]]
        pivotOffsetImageRatioXY = int ((savedOffset[0])*w), int ((savedOffset[1])*h)
        scaleBone =  int (rootBoone[0]-(pivotOffsetImageRatio[2]*15)*(pivotOffsetImageRatioXY[0]-rootBoone[0])), int (rootBoone[1]-(pivotOffsetImageRatio[2]*15)*(pivotOffsetImageRatioXY[1]-rootBoone[1]))
        diffXy=abs((np.subtract(pivotTracked,pivotOffsetImageRatioXY)))
        
        pivotTrackedWritst = int( result.multi_hand_landmarks[0].landmark[0].x*w), int( result.multi_hand_landmarks[0].landmark[0].y*h)
        pivotTrackedFirstBone = int( result.multi_hand_landmarks[0].landmark[1].x*w), int( result.multi_hand_landmarks[0].landmark[1].y*h)
        #print(np.dot(pivotTracked[1]-pivotTracked[0],pivotOffsetImageRatioXY[1]-pivotOffsetImageRatioXY[0]))
       
        #Compare gestures
        if(diffXy[0]<50 and diffXy[1]<50):
            recognizedColor =(0,255,0)
        #draw the circle for the tracked hand
        #print("draw saved gestures",id,pivotOffsetImageRatio[2]*30)
        cv2.circle(img, (pivotTracked), 5, (255, 0, 0)) 
        #draw the circle for the saved matching hand gesture
        cv2.circle(img ,pivotOffsetImageRatioXY,3, recognizedColor)
     
        return pivotOffsetImageRatio
#Init params
savedGesturesFilePath="handGestures.txt"
savedImageFilePath="Image.txt"
recognizedPoints = np.array([0,0,0])
savedOffsetPoints= np.array([0,0,0])
#savedCoords = readCoords(True, savedImageFilePath)
savedCoords = readCoords(True, savedGesturesFilePath)
recognizedColor = (0,0,255)
rootBoone = []

while True:    
    _,img = cap.read()
    result= hands.process(img)
   # if the gestrues should be compared this frame, start the compare method in the for loop
    recognizedPoints = []# clear the array before the loop starts to avoid overflow
    if result.multi_hand_landmarks:
        
        h, w, _ = img.shape
        savedOffsetPoints = []# clear the array before the loop starts to avoid overflow  
        rootBoone = result.multi_hand_landmarks[0].landmark[0]
        rootBoone = rootBoone.x*w, rootBoone.y*h, rootBoone.z
       
        
        # for each recognized point on the hand, get the id and the bone coordinates
        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark):   
            #calculate the saved handgesture in respect to the current hand position to make it independed of the image aspect
            savedOffsetPoints.append(drawSavedHandGesture(id,h,w, recognizedColor))
            #set the default points of the current handpostion
            recognizedPoints.append([lm.x, lm.y, lm.z])# append the circles from the hand to the array, 21 in total        
       
        # default hand drawing from puling
        #drawHandMediaPipe(img, lm)
        # Draw the result
    cv2.imshow("Hand tracking test", img)

    if quit():
        break
    #writeCoords(img)
    writeCoords(recognizedPoints, savedGesturesFilePath)
    readCoords(False, savedGesturesFilePath)  
  
        


TODO: 
*   Slider für Threshold etabielren, um genauigkeit der Gesten zu debuggen
*   Alex Ansatz einbauen und schauen, ob das erfassen der Fingerspitzen schon ausreicht nach dem Motto: 
    Nur die Handspitzen sind für anfänger, die korrekte Fingerhalteung ist dann das nächste Schwierig, its level
